## 撥放YouTube視訊
+ 改天pafy修正完畢後，下面安裝pafy指令 (先在Windows安裝git <a href="http://git-scm.com/download/win" target="_default" title="Git">Install git on Windows</a>再執行這份講義)

       !pip install git+https://github.com/Cupcakus/pafy
    
  可以改為
  
       !pip install pafy

In [6]:
try:
    import pafy
except:
    # 改天pafy修正完畢後，下面指令可以改為!pip install pafy
    # !pip install git+https://github.com/Cupcakus/pafy 
    # !pip install youtube_dl
    import pafy

try:
    import cv2
except:
    # !pip install opencv-python
    import cv2

url      = "https://www.youtube.com/watch?v=X0uaGjVUY_E&t=611s"
ty_video = pafy.new(url,basic=False,gdata=False)
best     = ty_video.getbest(preftype="mp4")
video    = cv2.VideoCapture(best.url)

if video is not None:
    fps    = int(video.get(cv2.CAP_PROP_FPS))
    width  = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print('Frame rate:',fps,'Frame width:',width,'Frame height:',height)
    
    frame_num = 0
    play_flag = 1
    total_frame=int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    def set_frame_number(x):
        global frame_num, play_flag
        if play_flag == 0:
            frame_num = x
            video.set(cv2.CAP_PROP_POS_FRAMES,frame_num)
        return
    
    def play(x):
        global play_flag
        play_flag = x
        return

    cv2.namedWindow('youtube')
    cv2.createTrackbar('frame no.','youtube',0,total_frame-1,set_frame_number)
    cv2.createTrackbar('play','youtube',0,1,play)
    cv2.setTrackbarPos('play','youtube',play_flag)
    
    while True:
        
        cv2.setTrackbarPos('frame no.','youtube',frame_num)
        
        if play_flag:
            grabbed, frame = video.read()
            cv2.imshow('youtube',frame)
            frame_num += 1
        
        k = cv2.waitKey(1000//fps)
        if k == 27:
            break
            
    cv2.destroyAllWindows()        
    video.release()
    
else:
    print('cannot open {}'.format(url))  

KeyError: 'dislike_count'

## 撥放YouTube直播

In [ ]:
try:
    import pafy
except:
    # 改天pafy修正完畢後，下面指令可以改為!pip install pafy
    !pip install git+https://github.com/Cupcakus/pafy 
    import pafy

try:
    import cv2
except:
    !pip install opencv-python
    import cv2

import numpy as np

try:
    from PIL import Image, ImageDraw, ImageFont
except:
    !pip install pillow
    from PIL import Image, ImageDraw, ImageFont
    
def cv2ImgAddText(img, text, pos, textColor=(0, 255, 0), textSize=20):
    if (isinstance(img, np.ndarray)):  #判斷是否OpenCV圖片類型
        img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img)
    fontText = ImageFont.truetype("C:\\Windows\Fonts\\simsun.ttc", textSize, encoding="utf-8")
    draw.text(pos, text, textColor, font=fontText)
    return cv2.cvtColor(np.asarray(img), cv2.COLOR_RGB2BGR)

urls = [
       ("https://www.youtube.com/watch?v=1OTZ9rjFv78",'石門水庫'),
       ("https://www.youtube.com/watch?v=z_mlibCfgFI",'三峽老街'),
       ("https://www.youtube.com/watch?v=UCG1aXVO8H8","台東多良"),
       ("https://www.youtube.com/watch?v=sZXBFjepdeQ","墾丁南灣"),
       ("https://www.youtube.com/watch?v=jFJ59-9tTyM","台東三仙台"),
       ("https://www.youtube.com/watch?v=dY2cRNr5Buw&list=PLgskA-LBVwZpd-xOnJgUqervKXJvQW6m7","阿里山"),
       ("https://www.youtube.com/watch?v=yeoV-wBdoxQ","台南黃金海岸"),
       ("https://www.youtube.com/watch?v=HmHKZ1Quvq8","日月潭"),
      ] 

videos = []
fps    = 100

for url in urls:
  try:  
      video = pafy.new(url[0])
      best = video.getbest(preftype="mp4")
      video = cv2.VideoCapture(best.url)
      if video is not None:
        fps    = min([fps,int(video.get(cv2.CAP_PROP_FPS))])
        width  = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
        print('Frame rate:',fps,'Frame width:',width,'Frame height:',height)
        videos.append((video,url[1]))    
  except:
      pass

if videos:
  cv2.namedWindow('youtube live')
  big_frame = np.zeros(((len(videos)+3)//4*180,(4 if len(videos)>=4 else len(videos))*320,3),dtype=np.uint8)
  while True:
    
    for i,video in enumerate(videos):
      grabbed, frame = video[0].read()
      frame = cv2.resize(frame,(320,180))
      frame = cv2ImgAddText(frame,video[1],(8,frame.shape[0]-24),(255, 255, 255))  
      row_idx = i//4
      col_idx = i%4
      big_frame[row_idx*frame.shape[0]:(row_idx+1)*frame.shape[0],
                col_idx*frame.shape[1]:(col_idx+1)*frame.shape[1],:] = frame[:,:,:]  
        
    cv2.imshow('youtube live',big_frame)
    k = cv2.waitKey(1000//fps)
    if k == 27:
        break

  cv2.destroyAllWindows()        
  for video in videos:
    video[0].release()
    
